In [1]:
# connect
import psycopg2
import json
import numpy as np
import time
# more connectstr
# "dbname='sqlbook' user='postgres' host='localhost' password='xxxx'"

#inputfile = open('connectstr', 'r')
inputfile = open('connucsd', 'r')
connstr = None

for i in inputfile:
    connstr = json.loads(i.strip())
    
inputfile.close()

conn = psycopg2.connect(connstr)    
conn.set_session(autocommit = True)
cur = conn.cursor();

In [2]:
def get_cust_item_list(cursor):
    freq = dict()
    items = []
    
    sql = """SELECT c.customerid, pr.asin
          FROM customers c, orders o, orderlines ol, products pr
          WHERE o.customerid = c.customerid
          and ol.orderid = o.orderid
          and pr.productid = ol.productid
          order by c.customerid """

    cursor.execute(sql)
    
    for i in cursor.fetchall():        
        if i[0] in freq:
            freq[i[0]].append(i[1])
        else:
            freq.update({i[0]:[i[1]]}) # item list per customer
        
        freq[i[0]] = list(set(freq[i[0]]))
        items.append(i[1]) # item list for all customers
    
    return freq, list(set(items))

In [3]:
freq, items = get_cust_item_list(cur)

In [4]:
def create_customer_matrix(freq, items):
    cols = len(items)
    lfreq = freq.keys()
    rows = len(lfreq)
    
    m = np.zeros((rows,cols))
    
    for r in range(rows):
        lst = freq[lfreq[r]] # get items per customer
        
        for i in lst: # go through each item
            m[r][items.index(i)] = 1 # update matrix
            
    return m

In [5]:
cust_item_matrix = create_customer_matrix(freq, items)

In [6]:
cust_item_matrix.shape

(189559, 3990)

In [7]:
def symmetric_matrix(matrix, items):
    rows, cols = matrix.shape
    m = np.zeros((0,cols)) # init matrix
    
    for i in range(cols):
        t = np.sum(matrix[matrix[:,i] > 0],axis=0)
        t[i] = 0
        m = np.vstack([m, t])
        
    return m

In [8]:
gen_matrix = symmetric_matrix(cust_item_matrix, items)

In [9]:
gen_matrix.shape

(3990, 3990)